<a href="https://colab.research.google.com/github/viliturpeinen/Colabs/blob/main/TEHTAVA6_KOE27112025_PAATOKSENTEKOMALLIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import itertools
import numpy as np

# Perustiedot

pinta_ala = 1200  # m2
yht_vastike = 6.5 * 12  # €/m2/vuosi
vuosi = 5

# Päätösmuuttujien vaihtoehdot

vuokratasot = [35, 65]       # €/m2/kk
korot = [0.06, 0.12]         # vuosikorko
inflaatiot = [0.02, 0.05]    # vuosittainen inflaatio

# Funktio kassavirralle vuodessa

def kassavirta(vuokra_m2, vastike, laina, korko, inflaatio, vuosi_num, alkuperainen_vuokra):
    """
    lasketaan kassavirta tälle vuodelle
    """
    # Vuokrien ja vastikkeen inflaatiokorjaus alkaen toisesta vuodesta
    if vuosi_num > 0:
        vuokra_m2 *= (1 + inflaatio) ** (vuosi_num)
    vuokratulot = vuokra_m2 * pinta_ala * 12  # €/vuosi
    # Vastikkeen maksaa vain tyhjistä tiloista
    tyhja_pinta = max(0, pinta_ala - pinta_ala)  # Tässä oletetaan 100% vuokraus
    vastikkeet = tyhja_pinta * vastike
    korkokulut = laina * korko
    return vuokratulot - vastikkeet - korkokulut

# Generoi kaikki kombinaatiot

kombinaatiot = list(itertools.product(vuokratasot, korot, inflaatiot))

# Funktio, joka testaa onko annettu velka mahdollinen kaikissa kombinaatioissa

def velka_ok(max_laina):
    for komb in kombinaatiot:
        vuokra, korko, inflaatio = komb
        laina = max_laina
        for v in range(vuosi):
            cf = kassavirta(vuokra, yht_vastike, laina, korko, inflaatio, v, vuokra)
            # Positiivinen kassavirta lyhentää lainaa
            if cf >= 0:
                laina -= cf
            else:  # Negatiivinen kassavirta lisää lainaa
                laina -= cf  # cf negatiivinen, laina kasvaa
            if laina < 0:
                laina = 0
            if laina > max_laina:
                return False
    return True

# Etsi optimivelka binäärihaulla

tolerance = 1e-2
low = 0
high = 1e7  # aloitetaan 10 miljoonasta
while high - low > tolerance:
    mid = (low + high)/2
    if velka_ok(mid):
        low = mid
    else:
        high = mid

print(f"Suurin mahdollinen optimivelka: {low:.2f} €")


Suurin mahdollinen optimivelka: 4200000.00 €
